In [ ]:
import pandas as pd
import matplotlib as plt

In [ ]:
import pandas as pd

df = pd.read_csv("E:/Hackathon/time_series_data.csv")
df['month'] = pd.to_datetime(df['month'])
df.set_index('month', inplace=True)
print(df.info())
print(df.head())


In [ ]:
print(df.isnull().sum())
df = df.interpolate(method='linear')  # Linear interpolation


In [ ]:
print(df.describe())
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
sns.boxplot(data=df)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set figure size
plt.figure(figsize=(12, len(df.columns) * 4))

# Create individual box plots for each numerical column
for i, column in enumerate(df.columns):
    plt.subplot(len(df.columns), 1, i + 1)
    sns.boxplot(x=df[column])
    plt.title(f'Box Plot of {column}')
    plt.xlabel(column)

plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from tsfresh.utilities.dataframe_functions import impute
from tsfresh import select_features

# Load tsfresh extracted features
extracted_features = pd.read_csv("E:/Hackathon/extracted_features (1).csv", index_col=0)  # or use directly if in memory

# Load target data (forecasted value year 3)
target_df = pd.read_csv("E:/Hackathon/target_data.csv")
target_series = target_df.set_index("client_id")["forecasted_value_year_3"]

# Step 1: Impute missing values in features
from tsfresh.utilities.dataframe_functions import impute
impute(extracted_features)

# Step 2: Align features and target
X = extracted_features.loc[target_series.index]
y = target_series.loc[X.index]

# Step 3: Feature selection (optional but recommended)
from tsfresh import select_features
X_selected = select_features(X, y)

# Step 4: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# Step 5: Train XGBoost
model = XGBRegressor(
    n_estimators=200,
    learning_rate=0.1,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

model.fit(X_train, y_train)

# Step 6: Evaluate
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"✅ RMSE: {rmse:.2f}")
print(f"✅ R² Score: {r2:.4f}")

In [ ]:
pip install tsfresh

In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from tsfresh.utilities.dataframe_functions import impute
from tsfresh import select_features

# Define targets
targets = ["forecasted_value_year_1", "forecasted_value_year_2", "forecasted_value_year_3"]

# Load data
extracted_features = pd.read_csv("E:/Hackathon/extracted_features (1).csv", index_col=0)
target_df = pd.read_csv("E:/Hackathon/target_data.csv")

# Prepare targets
target_df = target_df.set_index("client_id")
y_all = target_df[targets]

# Align features with targets
X = extracted_features.loc[y_all.index]
impute(X)

# Use year 3 for feature selection (proxy)
X_selected = select_features(X, y_all["forecasted_value_year_3"])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_selected, y_all, test_size=0.2, random_state=42)

# Define model
base_model = XGBRegressor(
    n_estimators=200,
    learning_rate=0.1,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

# Multi-output wrapper
multi_model = MultiOutputRegressor(base_model)
multi_model.fit(X_train, y_train)

# Predict and evaluate
y_pred = multi_model.predict(X_test)

for i, year in enumerate(targets):
    rmse = np.sqrt(mean_squared_error(y_test.iloc[:, i], y_pred[:, i]))
    r2 = r2_score(y_test.iloc[:, i], y_pred[:, i])
    print(f"\n🔮 {year}")
    print(f"✅ RMSE: {rmse:.2f}")
    print(f"✅ R² Score: {r2:.4f}")


In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from tsfresh.utilities.dataframe_functions import impute
from tsfresh import select_features

# Load data
extracted_features = pd.read_csv("E:/Hackathon/extracted_features (1).csv", index_col=0)
target_df = pd.read_csv("E:/Hackathon/target_data.csv")

# Define targets
targets = ["forecasted_value_year_1", "forecasted_value_year_2", "forecasted_value_year_3"]

for year in targets:
    print(f"\n🔮 Predicting {year}...")

    y = target_df.set_index("client_id")[year]

    # Align
    common_index = extracted_features.index.intersection(y.index)
    X = extracted_features.loc[common_index]
    y = y.loc[common_index]

    # Drop missing target values
    non_null_idx = y.dropna().index
    X = X.loc[non_null_idx]
    y = y.loc[non_null_idx]

    # Impute missing values in features
    impute(X)

    # Feature selection
    X_selected = select_features(X, y)

    # ⚠️ Skip if no features were selected
    if X_selected.shape[1] == 0:
        print(f"⚠️ No features selected for {year}. Skipping...")
        continue

    # Split
    X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

    # Train
    model = XGBRegressor(
        n_estimators=200,
        learning_rate=0.1,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )

    model.fit(X_train, y_train)

    # Evaluate
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    print(f"✅ RMSE: {rmse:.2f}")
    print(f"✅ R² Score: {r2:.4f}")


In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from tsfresh.utilities.dataframe_functions import impute

# Load data
extracted_features = pd.read_csv("E:/Hackathon/extracted_features (1).csv", index_col=0)
target_df = pd.read_csv("E:/Hackathon/target_data.csv")

# Define target columns
targets = ["forecasted_value_year_1", "forecasted_value_year_2", "forecasted_value_year_3"]

# Step 1: Clean features
# Drop columns with all NaNs or all zeros
cleaned_features = extracted_features.dropna(axis=1)  # Drop columns with NaNs
cleaned_features = cleaned_features.loc[:, (cleaned_features != 0).any(axis=0)]  # Drop all-zero columns

# Impute missing (in case anything slipped through)
impute(cleaned_features)

# Step 2: Loop through targets
for year in targets:
    print(f"\n🔮 Predicting {year}...")

    # Get target column and align with features
    y = target_df.set_index("client_id")[year]
    common_index = cleaned_features.index.intersection(y.index)
    X = cleaned_features.loc[common_index]
    y = y.loc[common_index]

    # Drop rows with missing target values
    valid_idx = y.dropna().index
    X = X.loc[valid_idx]
    y = y.loc[valid_idx]

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train model
    model = XGBRegressor(
        n_estimators=200,
        learning_rate=0.1,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )
    model.fit(X_train, y_train)

    # Evaluate
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    print(f"✅ RMSE: {rmse:.2f}")
    print(f"✅ R² Score: {r2:.4f}")


In [ ]:
import pandas as pd

# Load the dataset
file_path = (r"E:\Hackathon\time_series_data.csv")  # Replace with your actual file path
df = pd.read_csv(file_path)

# Define scenario adjustments
equity_impact = -0.078
fixed_income_impact = 0.066
macroeconomic_score_adjustment = -1.64
sentiment_index_adjustment = 0.11

# Apply adjustments
df["equity_allocation_pct"] *= (1 + equity_impact)
df["fixed_income_allocation_pct"] *= (1 + fixed_income_impact)
df["macroeconomic_score"] += macroeconomic_score_adjustment
df["sentiment_index"] += sentiment_index_adjustment

# Save the modified dataset
output_path = "adjusted_timeseries_data.csv"
df.to_csv(output_path, index=False)

print(f"Adjusted dataset saved to {output_path}")


In [ ]:
import pandas as pd
import numpy as np

# Load datasets
timeseries_file = r"E:\Hackathon\time_series_data.csv"  # Replace with actual file path
macro_file = r"E:\Hackathon\macro_scenarios.csv"  # Replace with actual file path

df = pd.read_csv(timeseries_file)
macro_df = pd.read_csv(macro_file)

# Select 1000 unique clients randomly (ensuring consistency across all scenarios)
unique_clients = df["client_id"].unique()
selected_clients = np.random.choice(unique_clients, 1000, replace=False)

# Filter dataset for selected clients
df_selected = df[df["client_id"].isin(selected_clients)]

# Process for each macroeconomic scenario
for index, scenario in macro_df.iterrows():
    scenario_id = scenario["scenario_id"]
    equity_impact = scenario["equity_impact"]
    fixed_income_impact = scenario["fixed_income_impact"]
    macroeconomic_score_adjustment = scenario["macroeconomic_score_adjustment"]
    sentiment_index_adjustment = scenario["sentiment_index_adjustment"]
    interest_rate_change = scenario["interest_rate_change"]

    # Create a copy of the filtered dataset to apply scenario adjustments
    df_adjusted = df_selected.copy()
    df_adjusted["equity_allocation_pct"] *= (1 + equity_impact)
    df_adjusted["fixed_income_allocation_pct"] *= (1 + fixed_income_impact)
    df_adjusted["macroeconomic_score"] += macroeconomic_score_adjustment
    df_adjusted["sentiment_index"] += sentiment_index_adjustment

    # Add new column for interest rate change
    df_adjusted["interest_rate_change"] = interest_rate_change

    # Save to CSV
    output_path = f"adjusted_timeseries_{scenario_id}.csv"
    df_adjusted.to_csv(output_path, index=False)
    print(f"Adjusted dataset for {scenario_id} saved to {output_path}")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor

# Load dataset
df = pd.read_csv(r"E:\Hackathon\engineered_stationary_data.csv")

# Drop original non-stationary raw features
non_stationary_raw_features = [
    'portfolio_value', 
    'equity_allocation_pct', 
    'fixed_income_allocation_pct', 
    'monthly_contribution', 
    'market_volatility_index', 
    'macroeconomic_score', 
    'sentiment_index'
]

# Target columns
target_cols = [
    'forecasted_value_year_1',
    'forecasted_value_year_2',
    'forecasted_value_year_3'
]

# Clean dataframe
df_cleaned = df.drop(columns=['client_id'] + non_stationary_raw_features)

# Drop non-numeric features (like date)
df_cleaned = df_cleaned.select_dtypes(include=[np.number])

# Separate X and y
X = df_cleaned.drop(columns=target_cols)
y = df_cleaned[target_cols]

# Random Forest Feature Selection
important_features = set()

for target in target_cols:
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X, y[target])
    
    importances = pd.Series(model.feature_importances_, index=X.columns)
    top_features = importances.sort_values(ascending=False).head(30).index.tolist()
    important_features.update(top_features)

# Keep only selected features
X_selected = X[list(important_features)]

print("✅ Top selected features:")
print(X_selected.columns.tolist())

In [ ]:
pip install tensorflow


In [ ]:
from tensorflow.keras.models import load_model
import tensorflow.keras.backend as K

# Define custom MSE function
def mse(y_true, y_pred):
    return K.mean(K.square(y_pred - y_true))

# Load the model with custom objects
custom_objects = {"mse": mse}
model = load_model(r"E:\Hackathon\lstm_portfolio_forecast_model.h5", custom_objects=custom_objects)

print("✅ Model loaded successfully!")


In [ ]:
import pandas as pd

# Load the dataset
file_path = r"E:\Hackathon\adjusted_timeseries_SCN_10.csv"  # Replace with your actual file path
df = pd.read_csv(file_path)

# Increase market_volatility_index by 30
df["market_volatility_index"] += 30

# Save the modified dataset
output_path = "adjusted_timeseries_SCN_10.csv"
df.to_csv(output_path, index=False)

print(f"Modified dataset saved to {output_path}")


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load dataset
df = pd.read_csv(r"E:\Hackathon\time_series_data.csv")
df = df.drop(columns=["client_id", "month"])

# Compute correlation matrix
corr_matrix = df.corr()

# Plot heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Feature Correlation Heatmap")
plt.show()
